[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/Copy_of_Copy_of_DNST_CIFAR10_AUGC1.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 20
num_filter = 25
compression = 0.8
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
from keras.preprocessing.image import ImageDataGenerator #, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True)
'''        vertical_flip=True,
        rotation_range=40,
        horizontal_flip=True
     fill_mode='nearest',
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
'''
datagen.fit(x_train)
train_gen = datagen.flow(x_train, y_train, batch_size)
test_gen = datagen.flow(x_test, y_test, batch_size)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
#num_filter = 12
#dropout_rate = 0.2
#l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [37]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 32, 32, 25)   675         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_129 (BatchN (None, 32, 32, 25)   100         conv2d_129[0][0]                 
__________________________________________________________________________________________________
activation_129 (Activation)     (None, 32, 32, 25)   0           batch_normalization_129[0][0]    
__________________________________________________________________________________________________
conv2d_130


dropout_180 (Dropout)           (None, 8, 8, 20)     0           conv2d_183[0][0]                 
__________________________________________________________________________________________________
concatenate_172 (Concatenate)   (None, 8, 8, 260)    0           concatenate_171[0][0]            
                                                                 dropout_180[0][0]                
__________________________________________________________________________________________________
batch_normalization_183 (BatchN (None, 8, 8, 260)    1040        concatenate_172[0][0]            
__________________________________________________________________________________________________
activation_183 (Activation)     (None, 8, 8, 260)    0           batch_normalization_183[0][0]    
__________________________________________________________________________________________________
conv2d_184 (Conv2D)             (None, 8, 8, 20)     46800       activation_183[0][0]             
_________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint
filepath= "weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [40]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
'''
model.fit_generator(train_gen,
                    epochs=epochs,
                    verbose=1,
                    steps_per_epoch=(len(x_train) // batch_size),
                    validation_data=test_gen,
                   callbacks=callbacks_list)

Epoch 1/50
781/781 [==============================] - 964s 1s/step - loss: 1.5869 - acc: 0.4204 - val_loss: 1.9470 - val_acc: 0.3943

Epoch 00001: val_acc improved from -inf to 0.39430, saving model to weights.best.hdf5
Epoch 2/50
 87/781 [==>...........................] - ETA: 13:02 - loss: 1.2742 - acc: 0.5388

781/781 [==============================] - 936s 1s/step - loss: 1.1528 - acc: 0.5869 - val_loss: 1.9442 - val_acc: 0.4993

Epoch 00002: val_acc improved from 0.39430 to 0.49930, saving model to weights.best.hdf5
Epoch 3/50
141/781 [====>.........................] - ETA: 12:01 - loss: 0.9674 - acc: 0.6510

781/781 [==============================] - 937s 1s/step - loss: 0.9306 - acc: 0.6682 - val_loss: 1.3477 - val_acc: 0.6059

Epoch 00003: val_acc improved from 0.49930 to 0.60590, saving model to weights.best.hdf5
Epoch 4/50
155/781 [====>.........................] - ETA: 11:46 - loss: 0.8381 - acc: 0.7070

781/781 [==============================] - 939s 1s/step - loss: 0.8129 - acc: 0.7138 - val_loss: 1.0934 - val_acc: 0.6672

Epoch 00004: val_acc improved from 0.60590 to 0.66720, saving model to weights.best.hdf5
Epoch 5/50
159/781 [=====>........................] - ETA: 11:39 - loss: 0.7452 - acc: 0.7404

781/781 [==============================] - 938s 1s/step - loss: 0.7267 - acc: 0.7468 - val_loss: 1.1462 - val_acc: 0.6625

Epoch 00005: val_acc did not improve from 0.66720
Epoch 6/50
185/781 [======>.......................] - ETA: 11:11 - loss: 0.6847 - acc: 0.7618

...

781/781 [==============================] - 937s 1s/step - loss: 0.3790 - acc: 0.8699 - val_loss: 0.6814 - val_acc: 0.7958

Epoch 00017: val_acc did not improve from 0.83350
Epoch 18/50
195/781 [======>.......................] - ETA: 11:03 - loss: 0.3503 - acc: 0.8801

781/781 [==============================] - 942s 1s/step - loss: 0.3598 - acc: 0.8745 - val_loss: 1.0159 - val_acc: 0.7551

Epoch 00018: val_acc did not improve from 0.83350
Epoch 19/50
195/781 [======>.......................] - ETA: 11:03 - loss: 0.3439 - acc: 0.8789

781/781 [==============================] - 941s 1s/step - loss: 0.3492 - acc: 0.8777 - val_loss: 0.6776 - val_acc: 0.7962

Epoch 00019: val_acc did not improve from 0.83350
Epoch 20/50
195/781 [======>.......................] - ETA: 11:00 - loss: 0.3283 - acc: 0.8841

654/781 [========================>.....] - ETA: 2:23 - loss: 0.3370 - acc: 0.8830

KeyboardInterrupt: ignored

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 22s 2ms/step
Test loss: 0.6434394887268543
Test accuracy: 0.8557


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')